In [35]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
for i in range(100):
    face_path = './dataset/train/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [20]:
face_origin.shape

(1080, 800, 3)

In [21]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [22]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][27:36]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [23]:
train_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        train_x[i][j] = globals()['norm'+str(i)][j]   

In [24]:
for i in range(100):
    face_path = './dataset/test/img/%s.png'%i
    face_origin = cv2.imread(face_path)
    face_copy = face_origin.copy()
    face_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    detector = dlib.get_frontal_face_detector()
    dlib_rects = detector(face_rgb, 1)
    
    img_how_rgb = cv2.cvtColor(face_copy, cv2.COLOR_BGR2RGB)
    
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    globals()['height'+str(i)] = face_origin.shape[0]
    
    globals()['img'+str(i)] = []
    
    for dlib_rect in dlib_rects:
        points = predictor(face_rgb, dlib_rect)
        list_points = list(map(lambda p: [p.x, p.y], points.parts()))
        globals()['img'+str(i)].append(list_points)
        globals()['img'+str(i)] = np.array(globals()['img'+str(i)])

In [25]:
for i in range(100):
    land = np.zeros((68,2))
    
    for j in range(68):
        land[j][0] = globals()['img'+str(i)][0][j][0]
        land[j][1] = globals()['img'+str(i)][0][j][1]
        
    x = land[:,0]
    y = land[:,1] * (-1) + globals()['height'+str(i)]
    
    x -= ((min(x) + max(x)) / 2)
    y -= ((min(y) + max(y)) / 2)
    
    norm_x = (x - min(x)) / (max(y)-min(y))
    norm_y = (y - min(y)) / (max(y)-min(y))
    
    globals()['norm'+str(i)] = np.column_stack((norm_x, norm_y))

In [26]:
# xyxyxyxyxyxyxy(0~16)
for i in range(100):
    globals()['norm'+str(i)] = globals()['norm'+str(i)][27:36]
    globals()['norm'+str(i)] = globals()['norm'+str(i)].flatten()

In [27]:
test_x = np.zeros((100, len(norm0)))
for i in range(100):
    for j in range(len(norm0)):
        test_x[i][j] = globals()['norm'+str(i)][j]   

In [28]:
train_y = pd.read_csv('./dataset/train/train.csv')
train_y = train_y.iloc[:, 22:30]
test_y = pd.read_csv('./dataset/test/test.csv')
test_y = test_y.iloc[:, 22:30]

In [12]:
# train_y = train_y.drop(['noseFlatten','nosePronounced','noseBridge','noseCurve'], axis=1)
# test_y = test_y.drop(['noseFlatten','nosePronounced','noseBridge','noseCurve'], axis=1)

In [29]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

model = LinearRegression()
model.fit(train_x, train_y)
# y_pred = model.predict(train_x)
# mae = mean_absolute_error(train_y, y_pred)

# mae

LinearRegression()

In [31]:
y_pred = model.predict(test_x)
mae = mean_absolute_error(test_y, y_pred)

mae

0.1240281411199708

In [14]:
#model.intercept_, model.coef_

In [20]:
# pd.set_option('display.max_rows', None)

In [32]:
prediction = pd.DataFrame(y_pred)
prediction.columns = ['noseSize','nosePosition', 'noseFlatten','nosePronounced','noseWidth', 'noseBridge', 'noseCurve', 'noseInclination']
# prediction.columns = ['noseSize', 'nosePosition','noseWidth','noseInclination']
prediction

,noseSize,nosePosition,noseFlatten,nosePronounced,noseWidth,noseBridge,noseCurve,noseInclination
0,0.378432,0.415773,0.493757,0.380154,0.320162,0.522676,0.521505,0.636121
1,0.318223,0.566885,0.491059,0.531520,0.358552,0.704679,0.383661,0.523399
2,0.354587,0.275652,0.644051,0.503630,0.574876,0.380413,0.498552,0.249376
3,0.512821,0.640112,0.400142,0.585253,0.348911,0.601845,0.557094,0.742090
4,0.349646,0.348959,0.552767,0.563734,0.487068,0.573305,0.453031,0.654462
...,...,...,...,...,...,...,...,...
95,0.656951,0.625304,0.322705,0.505799,0.467966,0.606989,0.526131,0.751394
96,0.449599,0.411613,0.470111,0.527597,0.364621,0.550303,0.484173,0.433681
97,0.587199,0.781408,0.548186,0.497231,0.386550,0.415069,0.577111,0.621739
98,0.626826,0.421727,0.396240,0.558801,0.772348,0.582270,0.391083,0.206927


In [33]:
test_y

,noseSize,nosePosition,noseFlatten,nosePronounced,noseWidth,noseBridge,noseCurve,noseInclination
0,0.528794,0.450309,0.204617,0.362828,0.274802,0.260083,0.561282,0.678466
1,0.235096,0.543748,0.319688,0.499098,0.544562,0.494427,0.490545,0.507280
2,0.400362,0.306366,0.752782,0.587933,0.600885,0.460328,0.622252,0.239924
3,0.539006,0.705693,0.551737,0.515948,0.380126,0.557269,0.291789,0.655423
4,0.434991,0.420111,0.604129,0.722225,0.600525,0.364166,0.531541,0.696418
...,...,...,...,...,...,...,...,...
95,0.574014,0.662468,0.242632,0.437827,0.438891,0.276147,0.233988,0.738699
96,0.335086,0.516407,0.258576,0.706496,0.402938,0.243166,0.505273,0.341421
97,0.377994,0.621198,0.213180,0.699112,0.250577,0.256895,0.588393,0.682466
98,0.486372,0.390966,0.262559,0.505704,0.792901,0.338591,0.532779,0.310331


In [34]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(test_y, prediction)**0.5
print(rmse)

0.15734066122553483
